In [1]:
import mmcv
from nuscenes.nuscenes import NuScenes
from PIL import Image
from nuscenes.utils.geometry_utils import view_points, box_in_image, BoxVisibility, transform_matrix
import json
import math
import os
import os.path as osp
import sys
import time
from datetime import datetime
from typing import Tuple, List, Iterable

import cv2
import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics
from PIL import Image
from matplotlib import rcParams
from matplotlib.axes import Axes
from pyquaternion import Quaternion
from tqdm import tqdm

import cv2
import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics
from PIL import Image
from matplotlib import rcParams
from matplotlib.axes import Axes
from pyquaternion import Quaternion
from tqdm import tqdm

from nuscenes.lidarseg.lidarseg_utils import colormap_to_colors, plt_to_cv2, get_stats, \
    get_labels_in_coloring, create_lidarseg_legend, paint_points_label
from nuscenes.panoptic.panoptic_utils import paint_panop_points_label, stuff_cat_ids, get_frame_panoptic_instances,\
    get_panoptic_instances_stats
from nuscenes.utils.data_classes import LidarPointCloud, RadarPointCloud, Box
from nuscenes.utils.data_io import load_bin_file, panoptic_to_lidarseg
from nuscenes.utils.geometry_utils import view_points, box_in_image, BoxVisibility, transform_matrix
from nuscenes.utils.map_mask import MapMask
from nuscenes.utils.color_map import get_colormap


try:
    # from waymo_open_dataset import dataset_pb2
    from simple_waymo_open_dataset_reader import WaymoDataFileReader
    from simple_waymo_open_dataset_reader import dataset_pb2, label_pb2
    from simple_waymo_open_dataset_reader import utils
except ImportError:
    raise ImportError(
        'Please run "pip install waymo-open-dataset-tf-2-1-0==1.2.0" '
        'to install the official devkit first.')
from pyquaternion import Quaternion
import mmcv
import numpy as np
import torch as tf
from glob import glob
from os.path import join
#from waymo_open_dataset.utils import range_image_utils, transform_utils
#from waymo_open_dataset.utils.frame_utils import parse_range_image_and_camera_projection
from pyquaternion import quaternion


In [8]:
gt = '/home/lizhiqi/bevformer/train.pkl'
#detr3d = '/home/lzq/workspace/bev/bevformer/test/detr3d_res101_gridmask/Tue_Dec_14_14_14_59_2021/pts_bbox/results_nusc.json'
#data1 = json.load(open(bevformer, 'r'))
gts = mmcv.load(gt)['infos']

In [9]:
frames = [each for each in gts]

In [10]:
frames

[{'frame_token': './data/waymo/waymo_format/training/segment-10017090168044687777_6380_000_6400_000_with_camera_labels.tfrecord',
  'lidar_path': '',
  'token': '',
  'sweeps': [],
  'cams': {'FRONT': {'data_path': './data/waymo/mv_waymo_format/training/image_0/0000000.png',
    'sensor2lidar_rotation': array([ 0.99995463, -0.00894421,  0.00286289, -0.00159672]),
    'sensor2lidar_translation': array([ 0.10914676, -0.02402951, -0.06822153]),
    'sensor2ego_translation': array([ 1.53914676, -0.02402951,  2.11577847]),
    'sensor2ego_rotation': array([ 0.99995463, -0.00894421,  0.00286289, -0.00159672]),
    'ego2global_translation': array([-1257.18321084, 10546.04168252,    22.447     ]),
    'ego2global_rotation': array([ 0.98697778, -0.00427515,  0.10863356,  0.1185552 ]),
    'cam_intrinsic': array([[ 2.05961201e+03,  2.05961201e+03,  9.52412190e+02],
           [ 6.34587208e+02,  3.54528738e-02, -3.38300854e-01],
           [ 1.92295962e-05,  7.13855107e-04,  0.00000000e+00]]),
  

In [12]:
q = Quaternion([-0.70630791,  0.00339895, -0.00157453,  0.70789484])

In [15]:
q.inverse.yaw_pitch_roll

(1.5730315031944948, 0.002587995234111671, 0.007030695331067703)

In [16]:
gt2 = '/home/lizhiqi/bevformer/data/nuscenes/nuscenes_infos_val.pkl'

In [17]:
gts2 = mmcv.load(gt2)['infos']

In [18]:
gts2[0]

{'lidar_path': './data/nuscenes/samples/LIDAR_TOP/n015-2018-08-02-17-16-37+0800__LIDAR_TOP__1533201470448696.pcd.bin',
 'token': 'fd8420396768425eabec9bdddf7e64b6',
 'sweeps': [],
 'cams': {'CAM_FRONT': {'data_path': './data/nuscenes/samples/CAM_FRONT/n015-2018-08-02-17-16-37+0800__CAM_FRONT__1533201470412460.jpg',
   'type': 'CAM_FRONT',
   'sample_data_token': 'b8fba7d78cf547b996c431dec1f5ee26',
   'sensor2ego_translation': [1.70079118954, 0.0159456324149, 1.51095763913],
   'sensor2ego_rotation': [0.4998015430569128,
    -0.5030316162024876,
    0.4997798114386805,
    -0.49737083824542755],
   'ego2global_translation': [249.90018228466124, 917.5514452794962, 0.0],
   'ego2global_rotation': [0.9984397071954904,
    -0.008652973321832425,
    0.002523596051210081,
    -0.05510815376190409],
   'timestamp': 1533201470412460,
   'sensor2lidar_rotation': array([[ 0.99996937,  0.0067556 , -0.0039516 ],
          [ 0.00382456,  0.01871645,  0.99981752],
          [ 0.00682833, -0.99980201

In [2]:

dataset = WaymoDataFileReader('/home/lizhiqi/bevformer/data/waymo/waymo_format/training/segment-10017090168044687777_6380_000_6400_000_with_camera_labels.tfrecord')

In [3]:
for frame in dataset:
    break

In [6]:
frame.context.camera_calibrations[1].intrinsic

[2046.6346113297673, 2046.6346113297673, 975.056043638034, 640.9087071643551, 0.030496309445083532, -0.3101716556652957, 0.002555391421798901, 0.0008861465365293735, 0.0]

In [39]:
import mmcv
gt = mmcv.load('data/waymo/kitti_format/waymo_infos_train.pkl')
for each in gt:
    r2 = each['pose'][:3, :3]
    print(Quaternion(matrix=r2).yaw_pitch_roll)

In [40]:
from nuscenes.eval.common.utils import quaternion_yaw, Quaternion

(0.2948141973114111, 0.20291360833632938, -0.05368495968330055)
(0.2694061379610221, 0.2124440065487469, -0.03943245584774899)


In [46]:
import numpy as np
0.2/np.pi*180

11.459155902616464

In [41]:
len(gt)

794

In [47]:
r = Quaternion(matrix=r2)

In [48]:
dir(r)

['__abs__',
 '__add__',
 '__bool__',
 '__class__',
 '__complex__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__float__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__idiv__',
 '__imatmul__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ipow__',
 '__isub__',
 '__itruediv__',
 '__le__',
 '__lt__',
 '__matmul__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__pow__',
 '__radd__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmatmul__',
 '__rmul__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_fast_normalise',
 '_from_axis_angle',
 '_from_matrix',
 '_normalise',
 '_q_bar_matrix',
 '_q_matrix',
 '_rotate_quaternion',
 '_sum_of_squares',
 '_validate_number_sequ

In [50]:
r

Quaternion(0.9854452512665322, -0.00519062590859225, 0.10767423155772447, 0.13144190358242608)

In [51]:
s = np.zeros([4])

In [53]:
s[:] = r

In [54]:
s

array([0.98544525, 0.98544525, 0.98544525, 0.98544525])